In [15]:
if 'g' in locals():
    driver = g.driver
    g = CoinGame(driver=driver)
else:
    g = CoinGame()
    driver = g.driver

Cheaters are only biased against heads!

In [10]:
g.reset_data()
iterations = 0
with open('log.txt', 'w') as f:
    f.write('Score, Flips left, Heads, Tails, Difference\n')

In [21]:
flips_left = None
while flips_left is None or flips_left > 0:
    iterations += 1
    data = g.get_data()
    n_heads = data['heads']
    n_tails = data['tails']
    flips_left = data['flips_left']
    n_throws = n_heads + n_tails
    if n_throws == 0:
        g.flip_five_coins()
    diff = (n_heads-n_tails)
    with open('log.txt', 'a') as f:
        f.write(f'{data["score"]}, {flips_left}, {n_heads}, {n_tails}, {diff}\n')
    if diff > 4:
        g.label_cheater()
        time.sleep(1)
    elif diff < -3 or n_throws > 10:
        g.label_fair()
        time.sleep(1)
    g.flip_one_coin()

KeyboardInterrupt: 

In [ ]:
import pandas as pd
pd.read_csv("logs.txt")